In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers==3.0.2
!pip install OpenHowNet==0.0.1a11
!pip install nltk==3.5
#import transformers1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 769 kB 20.5 MB/s 
     |████████████████████████████████| 3.0 MB 61.3 MB/s 
     |████████████████████████████████| 880 kB 69.2 MB/s 
     |████████████████████████████████| 1.3 MB 32.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=80f34698386f743480416d15266585b8204acb318c1dc2dcc6b555032bfd12f0
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 621 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 34.5 MB/s 
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl

In [ ]:
import torch
from transformers import BertTokenizer, BertModel, AdamW, BertSememeModel
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import os
import time
import random
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
random.seed(42)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch = 2
sample_num = 19
learning_rate = 3e-5
epochs = 40
max_length = 80

In [ ]:
def load_data(path):
    with open(path, 'r', encoding='utf-8') as f:
        former, middle, latter = [], [], []

        lines = f.readlines()
        for line in lines:
            line = line.strip().lower().split('\t')
            former.append(line[0])
            middle.append(line[1])
            latter.append(line[2])
        
        train_former, valid_former, test_former = former[:1000], former[1000:2000], former[2000:3000]
        train_middle, valid_middle, test_middle = middle[:1000], middle[1000:2000], middle[2000:3000]
        train_latter, valid_latter, test_latter = latter[:1000], latter[1000:2000], latter[2000:3000]

    all_quotes = train_middle + valid_middle + test_middle
    all_quotes = sorted(list(set(all_quotes)))

    y_train = [all_quotes.index(q) for q in train_middle]
    y_valid = [all_quotes.index(q) for q in valid_middle]
    y_test = [all_quotes.index(q) for q in test_middle]

    trains = [train_former, train_middle, train_latter]
    valids = [valid_former, valid_middle, valid_latter]
    tests = [test_former, test_middle, test_latter]
    y = [torch.LongTensor(y_train), torch.LongTensor(y_valid), torch.LongTensor(y_test)]

    return trains, valids, tests, y , all_quotes

In [ ]:
data_path = "/content/drive/MyDrive/quoter/data/modern_chinese.txt"
trains, valids, tests, y, all_quotes = load_data(data_path)

# get the Tokenizer used for pretraining model
PRETRAINED_MODEL_NAME = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)


In [ ]:
# Obtained directly from the source code
def make_context_tensors(former, latter):
    input_ids = []
    token_type_ids = []
    attention_masks = []
    mask_ids = []
    for f, l in zip(former, latter):
        sent = f + "[MASK]" + l
        encoded_dict = tokenizer.encode_plus(sent,
                                             add_special_tokens=True,
                                             max_length=150,
                                             pad_to_max_length=True,
                                             truncation=True,
                                             return_attention_mask=True,
                                             return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'])
        token_type_ids.append(encoded_dict['token_type_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        mask_index = encoded_dict['input_ids'][0].tolist().index(103)
        mask_ids.append(mask_index)
    input_ids = torch.cat(input_ids, dim=0)
    token_type_ids = torch.cat(token_type_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    return input_ids, token_type_ids, attention_masks, torch.LongTensor(mask_ids)

In [ ]:
train_input_ids, train_token_type_ids, train_attention_masks, train_mask_ids = make_context_tensors(trains[0], trains[2])
valid_input_ids, valid_token_type_ids, valid_attention_masks, valid_mask_ids = make_context_tensors(valids[0], valids[2])

In [ ]:
# Obtained directly from source code
class Dataset(Dataset):
    def __init__(self, input_ids, token_type_ids, attention_masks, mask_ids,
                 quote):
        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_masks = attention_masks
        self.mask_ids = mask_ids
        self.quote = quote

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        if self.quote is None:
            return self.input_ids[idx], self.token_type_ids[
                idx], self.attention_masks[idx], self.mask_ids[idx]
        return self.input_ids[idx], self.token_type_ids[
            idx], self.attention_masks[idx], self.mask_ids[idx], self.quote[
                idx]


train_dataset = Dataset(input_ids=train_input_ids,
                        token_type_ids=train_token_type_ids,
                        attention_masks=train_attention_masks,
                        mask_ids=train_mask_ids,
                        quote=trains[1])
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch,
                          shuffle=True,
                          num_workers=2)
valid_dataset = Dataset(input_ids=valid_input_ids,
                        token_type_ids=valid_token_type_ids,
                        attention_masks=valid_attention_masks,
                        mask_ids=valid_mask_ids,
                        quote=valids[1])
valid_loader = DataLoader(dataset=valid_dataset,
                          batch_size=batch,
                          shuffle=True,
                          num_workers=2)


In [ ]:
def generate_quotes(quote, num):
    quotes_selcet = all_quotes[:]
    quotes_selcet.remove(quote)
    quotes = random.sample(quotes_selcet, num)
    quotes.append(quote)
    random.shuffle(quotes)
    return quotes

# Obtained directly from the source code
def make_quote_tensors(quote):
    quotes = generate_quotes(quote, num=sample_num)
    label = quotes.index(quote)
    input_ids = []
    for q in quotes:
        encoded_dict = tokenizer.encode_plus(q,
                                             add_special_tokens=True,
                                             max_length=max_length,
                                             pad_to_max_length=True,
                                             truncation=True,
                                             return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'])
    input_ids = torch.cat(input_ids, 0)  # [num, 80]
    quote_ids = torch.LongTensor([all_quotes.index(q) for q in quotes])
    return input_ids, label, quote_ids


In [ ]:
class Context_Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert_model = BertModel.from_pretrained(PRETRAINED_MODEL_NAME)
        self.dropout = nn.Dropout(0.5)

    def forward(self, context_input_ids, context_token_type_ids, context_attention_masks, mask_ids):
        outputs = self.bert_model(input_ids=context_input_ids,
                                  token_type_ids=context_token_type_ids,
                                  attention_mask=context_attention_masks)
        
        last_hidden_state = outputs[0]
        all_context = []
        for i in range(len(last_hidden_state)):
            mask = last_hidden_state[i][mask_ids[i]]
            mask = self.dropout(mask)
            context = mask.unsqueeze(dim=0)
            all_context.append(context)

        return torch.cat(all_context, dim=0)


class Quote_Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert_model = BertSememeModel.from_pretrained(PRETRAINED_MODEL_NAME)
        #self.bert_model = BertModel.from_pretrained(PRETRAINED_MODEL_NAME)
        # self.dropout = nn.Dropout(0.5)

    def forward(self, quotes):
        quote_tensor = []
        labels = []
        for quote in quotes:
            quote_input_ids, label, quote_ids = make_quote_tensors(quote)
            quote_input_ids = quote_input_ids.to(device)
            quote_ids = quote_ids.to(device)
            outputs = self.bert_model(input_ids=quote_input_ids)
            output = torch.mean(outputs[0], dim=1)
            
            quote_tensor.append(output)
            labels.append(label)
        quote_tensor = torch.stack(quote_tensor, dim=0)
        return quote_tensor, labels


class QuotRec_Net(nn.Module):
    def __init__(self, context_model, quote_model):
        super().__init__()
        self.context_model = context_model
        self.quote_model = quote_model

    def forward(self, input_ids, token_type_ids, attention_masks, mask_ids,
                quotes):
        context_output = self.context_model(input_ids, token_type_ids,
                                           attention_masks, mask_ids)
        context_output = context_output.unsqueeze(dim=1)

        quote_output, labels = self.quote_model(quotes)
        quote_output = quote_output.permute(0, 2, 1)

        outputs = torch.matmul(context_output, quote_output).squeeze(dim=1)
        return outputs, torch.LongTensor(labels)

context_model = Context_Encoder()
quote_model = Quote_Encoder()
model = QuotRec_Net(context_model, quote_model)
model.to(device)

You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


QuotRec_Net(
  (context_model): Context_Encoder(
    (bert_model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(21128, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)

In [ ]:
import math
import tqdm.notebook as tq
def training(model, epoch, train, valid, device):

    len_train = len(train)
    len_valid = len(valid)

    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_acc = 0
    best_loss = math.inf
    losses = []
    count = 0

    for epoch in tq.tqdm(range(epoch)):
        start = time.perf_counter()
        total_loss, total_acc = 0, 0
        print("Epoch: ", epoch + 1)

        model.train()
        for i, (input_ids, token_type_ids, attention_masks, mask_ids, quotes) in enumerate(train):
            input_ids = input_ids.to(device)
            token_type_ids = token_type_ids.to(device)
            attention_masks = attention_masks.to(device)
            mask_ids = mask_ids.to(device, dtype=torch.long)
            
            optimizer.zero_grad()
            outputs, labels = model(input_ids, token_type_ids, attention_masks,mask_ids, quotes)
            labels = labels.to(device, dtype=torch.long)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            _, pred = torch.max(outputs.cpu().data, 1)
            acc = accuracy_score(pred, labels.cpu())
            total_loss += loss.item()
            total_acc += acc
        print('Train | Loss:{:.5f} Acc:{:.3f}'.format(total_loss, total_acc / len_train))

        model.eval()
        with torch.no_grad():
            total_loss, total_acc = 0, 0
            for i, (input_ids, token_type_ids, attention_masks, mask_ids, quotes) in enumerate(valid):
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                mask_ids = mask_ids.to(device, dtype=torch.long)

                outputs, labels = model(input_ids, token_type_ids,attention_masks, mask_ids, quotes)
                labels = labels.to(device, dtype=torch.long)
                loss = criterion(outputs, labels)
                _, pred = torch.max(outputs.cpu().data, 1)

                acc = accuracy_score(pred, labels.cpu())
                total_loss += loss.item()
                total_acc += acc
            losses.append(total_loss)
            print('Valid | Loss:{:.5f} Acc:{:.3f}'.format(total_loss, total_acc / len_valid))

            if total_acc >= best_acc and total_loss <= best_loss:
                best_acc = total_acc
                best_loss = total_loss
                if not os.path.exists("./model"):
                    os.mkdir("./model")
                torch.save(model.quote_model.state_dict(), "/content/drive/MyDrive/model/modern_chinese_quote.pth")
                torch.save(model.context_model.state_dict(), "/content/drive/MyDrive/model/modern_chinese_context.pth")
                count = 0
            elif total_loss > best_loss:
                count += 1

        end = time.perf_counter()

        if count == 3:
            print("Early Stopping")
            break


training(model=model,
         epoch=epochs,
         train=train_loader,
         valid=valid_loader,
         device=device)


def make_tensors(quotes):
    input_ids = []
    for q in quotes:
        encoded_dict = tokenizer.encode_plus(q,
                                             add_special_tokens=True,
                                             max_length=max_length,
                                             pad_to_max_length=True,
                                             truncation=True,
                                             return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'])
    input_ids = torch.cat(input_ids, 0)
    return input_ids


quote_input_ids = make_tensors(all_quotes)


  0%|          | 0/40 [00:00<?, ?it/s]

Epoch:  1
Train | Loss:1120.10286 Acc:0.335
Valid | Loss:1127.82479 Acc:0.402
Epoch:  2
Train | Loss:789.08613 Acc:0.534
Valid | Loss:1038.74296 Acc:0.424
Epoch:  3
Train | Loss:516.63204 Acc:0.687
Valid | Loss:1281.50861 Acc:0.385
Epoch:  4
Train | Loss:339.04292 Acc:0.789
Valid | Loss:1389.68664 Acc:0.439
Epoch:  5
Train | Loss:303.00448 Acc:0.827
Valid | Loss:1505.68285 Acc:0.432
Early Stopping


In [ ]:
# Generate sentence vector for quotes
quote_model = BertModel.from_pretrained(PRETRAINED_MODEL_NAME)
model_dict = quote_model.state_dict()
save_model_state = torch.load("/content/drive/MyDrive/model/modern_chinese_quote.pth")

state_dict = {k[11:]: v for k, v in save_model_state.items() if k[11:] in model_dict.keys()}
model_dict.update(state_dict)
quote_model.load_state_dict(model_dict)

quote_model = quote_model.to(device)
quote_input_ids = quote_input_ids.to(device)

quote_embeddings = []
quote_model.eval()

with torch.no_grad():
    for input_ids in quote_input_ids:
        input_ids = input_ids.unsqueeze(dim=0)
        outputs = quote_model(input_ids=input_ids)
        quote_tensor = torch.mean(outputs[0], dim=1)
        quote_embeddings.append(quote_tensor)
    quote_embeddings = torch.cat(quote_embeddings, dim=0)


In [ ]:
# Use the mask method for training
class QuotRecNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert_model = BertModel.from_pretrained(PRETRAINED_MODEL_NAME)
        self.dropout = nn.Dropout(0.5)

    def forward(self, input_ids, token_type_ids, attention_masks,
                mask_ids, quote_tensor):
        outputs = self.bert_model(input_ids=input_ids,
                                  token_type_ids=token_type_ids,
                                  attention_mask=attention_masks)
        last_hidden_state = outputs[0]
        all_outputs = []
        for i in range(len(last_hidden_state)):
            mask = last_hidden_state[i][mask_ids[i]]
            context = self.dropout(mask)
            context = context.unsqueeze(dim=0)
            output = torch.mm(context, quote_tensor.t())
            all_outputs.append(output)
        all_outputs = torch.cat(all_outputs, dim=0)
        return all_outputs


model = QuotRecNet()
model.to(device)


QuotRecNet(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [ ]:
# Evaluation Metrics
# Rank
def rank_gold(predicts, golds):
  ranks = []
  ps = predicts.data.cpu().numpy()
  gs = golds.cpu().numpy()
  for i in range(len(ps)):
    predict = ps[i]
    gold_index = gs[i]
    predict_value = predict[gold_index]
    predict_sort = sorted(predict, reverse=True)
    predict_index = predict_sort.index(predict_value)
    if predict_index == -1:
        break
    ranks.append(predict_index)
  return ranks


# NDCG@5
def get_NDCG(ranks):
    total = 0.0
    for r in ranks:
        if r < 5:  # k=5
            total += 1.0 / np.log2(r + 2)
    return total / len(ranks)


# get recall@k
def recall(predicts, golds):
    predicts = predicts.data.cpu().numpy()
    golds = list(golds)
    predicts_index = list(np.argsort(-predicts, axis=1))
    predicts_index = [list(element) for element in predicts_index]
    recall_values = [0, 0, 0, 0, 0, 0, 0] # 1, 3, 5, 10, 20, 30, 100, 300, 500
    recalls = [1, 3, 5, 10, 20, 30, 100]

    for i in range(len(golds)):
        gold_value_index = predicts_index[i].index(golds[i])
        for val in range(len(recalls)):
            if gold_value_index < recalls[val]:
                recall_values[val] += 1

    return recall_values

def get_mrr(ranks):
    return np.average([1.0 / (r + 1) for r in ranks])

In [ ]:
def training_mask(model, epoch, train, valid, quote_tensor, device):
    learning_rate = 3e-5
    len_train = len(train)
    len_valid = len(valid)

    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_MRR = 0
    count = 0
    quote_tensor = quote_tensor.to(device)
    for epoch in range(epoch):
        start = time.perf_counter()
        print("Epoch: ", epoch + 1)
        total_loss, total_MRR, total_NDCG = 0, 0, 0
        
        model.train()
        for i, (input_ids, token_type_ids, attention_masks, mask_ids, labels) in enumerate(train):
            input_ids = input_ids.to(device)
            token_type_ids = token_type_ids.to(device)
            attention_masks = attention_masks.to(device)
            mask_ids = mask_ids.to(device, dtype=torch.long)
            labels = labels.to(device, dtype=torch.long)

            optimizer.zero_grad()
            outputs = model(input_ids, token_type_ids, attention_masks, mask_ids, quote_tensor)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
  
            ranks = rank_gold(outputs, labels)
            MRR = np.average([1.0 / (r + 1) for r in ranks])
            NDCG = get_NDCG(ranks)
            total_loss += loss.item()
            total_MRR += MRR
            total_NDCG += NDCG
        end = time.perf_counter()
        print('Epoch running time :{:.0f}'.format(end - start))
        print('Train | Loss:{:.3f} MRR: {:.3f} NDCG: {:.3f}'.format(total_loss, total_MRR/len_train, total_NDCG/len_train))

        # validation
        model.eval()
        with torch.no_grad():
            total_loss, total_MRR, total_NDCG = 0, 0, 0
            for i, (input_ids, token_type_ids, attention_masks, mask_ids, labels) in enumerate(valid):
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                mask_ids = mask_ids.to(device, dtype=torch.long)
                labels = labels.to(device, dtype=torch.long)
                outputs = model(input_ids, token_type_ids, attention_masks, mask_ids, quote_tensor)
                loss = criterion(outputs, labels)
                
                ranks = rank_gold(outputs, labels)
                MRR = get_mrr(ranks)
                NDCG = get_NDCG(ranks)
                
                total_loss += loss.item()
                total_MRR += MRR
                total_NDCG += NDCG
            print("Valid | Loss:{:.5f} MRR: {:.3f} NDCG: {:.3f}".format(total_loss, total_MRR / len_valid,total_NDCG / len_valid))
        
        if total_MRR > best_MRR:
            best_MRR = total_MRR
            torch.save(model, "/content/drive/MyDrive/model/model_modern_chinese.model")
            count = 0
        else:
            learning_rate = learning_rate * 0.9
            count += 1
        
        # Early Stopping
        if count == 3:
            break

In [ ]:
# Mask Dataset and DataLoader
class Dataset_Mask(Dataset):

    def __init__(self, input_ids, token_type_ids, attention_masks, mask_ids,
                 y):
        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_masks = attention_masks
        self.mask_ids = mask_ids
        self.label = y

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        if self.label is None:
            return self.input_ids[idx], self.token_type_ids[
                idx], self.attention_masks[idx], self.mask_ids[idx]
        return self.input_ids[idx], self.token_type_ids[
            idx], self.attention_masks[idx], self.mask_ids[idx], self.label[
                idx]


print("loading train and valid dataloader ...")
train_dataset_mask = Dataset_Mask(input_ids=train_input_ids,
                                  token_type_ids=train_token_type_ids,
                                  attention_masks=train_attention_masks,
                                  mask_ids=train_mask_ids,
                                  y=y[0])
train_loader_mask = DataLoader(dataset=train_dataset_mask,
                               batch_size=batch,
                               shuffle=True,
                               num_workers=2)
valid_dataset_mask = Dataset_Mask(input_ids=valid_input_ids,
                                  token_type_ids=valid_token_type_ids,
                                  attention_masks=valid_attention_masks,
                                  mask_ids=valid_mask_ids,
                                  y=y[1])
valid_loader_mask = DataLoader(dataset=valid_dataset_mask,
                               batch_size=batch,
                               shuffle=True,
                               num_workers=2)
print("start traing......")
training_mask(model=model,
              epoch=epochs,
              train=train_loader_mask,
              valid=valid_loader_mask,
              quote_tensor=quote_embeddings,
              device=device)

loading train and valid dataloader ...
start traing......
Epoch:  1
Epoch running time :45
Train | Loss:2376.683 MRR: 0.229 NDCG: 0.231
Valid | Loss:2765.51097 MRR: 0.185 NDCG: 0.185
Epoch:  2
Epoch running time :45
Train | Loss:1403.334 MRR: 0.521 NDCG: 0.543
Valid | Loss:2608.93168 MRR: 0.244 NDCG: 0.245
Epoch:  3
Epoch running time :45
Train | Loss:797.376 MRR: 0.755 NDCG: 0.778
Valid | Loss:2687.81166 MRR: 0.244 NDCG: 0.241
Epoch:  4
Epoch running time :45
Train | Loss:463.578 MRR: 0.887 NDCG: 0.902
Valid | Loss:2664.60437 MRR: 0.265 NDCG: 0.268
Epoch:  5
Epoch running time :45
Train | Loss:308.447 MRR: 0.936 NDCG: 0.947
Valid | Loss:2646.29110 MRR: 0.266 NDCG: 0.266
Epoch:  6
Epoch running time :45
Train | Loss:205.262 MRR: 0.968 NDCG: 0.976
Valid | Loss:2688.18826 MRR: 0.266 NDCG: 0.267
Epoch:  7
Epoch running time :45
Train | Loss:136.756 MRR: 0.985 NDCG: 0.989
Valid | Loss:2685.19742 MRR: 0.277 NDCG: 0.277
Epoch:  8
Epoch running time :45
Train | Loss:103.008 MRR: 0.986 NDCG: 0

In [ ]:
def test(model, test_loader, quote_tensor, device):
    model.eval()
    t_batch = len(test_loader)
    criterion = nn.CrossEntropyLoss()
    quote_tensor = quote_tensor.to(device)
    with torch.no_grad():
        total_loss, total_MRR, total_NDCG, total_ranks = 0, 0, 0, 0
        total_recalls = [0, 0, 0, 0, 0, 0, 0]
        all_ranks = []
        for i, (input_ids, token_type_ids, attention_masks, mask_ids, labels) in enumerate(test_loader):
            input_ids = input_ids.to(device)
            token_type_ids = token_type_ids.to(device)
            attention_masks = attention_masks.to(device)
            mask_ids = mask_ids.to(device, dtype=torch.long)
            labels = labels.to(device, dtype=torch.long)
            
            outputs = model(input_ids, token_type_ids, attention_masks, mask_ids, quote_tensor)
            loss = criterion(outputs, labels)
            
            ranks = rank_gold(outputs, labels)
            all_ranks += ranks
            MRR = np.average([1.0 / (r + 1) for r in ranks])
            NDCG = get_NDCG(ranks)
            recalls = recall(outputs, labels)
            
            total_loss += loss.item()
            total_MRR += MRR
            total_NDCG += NDCG
            total_ranks += np.sum(ranks)
            total_recalls = [x + y for x, y in zip(total_recalls, recalls)]

        total_recalls = [element / len(y[2]) for element in total_recalls]

        print(
            "Test | Loss:{:.5f} MRR: {:.3f} NDCG: {:.3f} Mean Rank: {:.0f} Median Rank: {:.0f} Variance: {:.0f}"
            .format(total_loss, total_MRR / t_batch,
                    total_NDCG / t_batch, np.mean(all_ranks),
                    np.median(all_ranks)+1,
                    np.std(all_ranks)))
        print("Recall@[1,3,5,10,20,30,100]: " + str(total_recalls))
        

test_input_ids, test_token_type_ids, test_attention_masks, test_mask_ids = make_context_tensors(tests[0], tests[2])
test_dataset_mask = Dataset_Mask(input_ids=test_input_ids,
                                 token_type_ids=test_token_type_ids,
                                 attention_masks=test_attention_masks,
                                 mask_ids=test_mask_ids,
                                 y=y[2])
test_loader_mask = DataLoader(dataset=test_dataset_mask,
                              batch_size=batch,
                              num_workers=2)

model = torch.load('/content/drive/MyDrive/model/model_modern_chinese.model')
model.to(device)
test(model=model,
     test_loader=test_loader_mask,
     quote_tensor=quote_embeddings,
     device=device)

Test | Loss:2790.86053 MRR: 0.273 NDCG: 0.275 Mean Rank: 143 Median Rank: 30 Variance: 227
Recall@[1,3,5,10,20,30,100]: [0.208, 0.304, 0.333, 0.391, 0.444, 0.503, 0.656]
